In [45]:
import cv2
import os
import rosbag2_py

from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from deepracer_interfaces_pkg.msg import InferResultsArray, InferResults
from rclpy.serialization import deserialize_message

from matplotlib import pyplot as plt
import pandas as pd

In [46]:
bridge = CvBridge()

In [47]:
def get_rosbag_options(path, serialization_format='cdr'):
    storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')

    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format=serialization_format,
        output_serialization_format=serialization_format)

    return storage_options, converter_options

In [48]:
bag_path = '/workspaces/deepracer-devel/output/reinvent_logs/deepracer-bag-20221201-165534'

In [49]:
storage_options, converter_options = get_rosbag_options(bag_path)

reader = rosbag2_py.SequentialReader()
reader.open(storage_options, converter_options)

[INFO] [1673472533.427147179] [rosbag2_storage]: Opened database '/workspaces/deepracer-devel/output/reinvent_logs/deepracer-bag-20221201-165534/deepracer-bag-20221201-165534_0.db3' for READ_ONLY.


In [50]:
storage_filter = rosbag2_py.StorageFilter(topics=['/inference_pkg/rl_results'])
reader.set_filter(storage_filter)

first_stamp = -1
steps_data = {'steps': []}

while reader.has_next():
    step = {}
    
    (topic, data, t) = reader.read_next()
    msg = deserialize_message(data, InferResultsArray)

    # Timestamp
    timestamp: float = (msg.images[0].header.stamp.sec + msg.images[0].header.stamp.nanosec / 1e9)

    if first_stamp == -1:
        first_stamp = timestamp
        timestamp = 0
    else:
        timestamp = timestamp - first_stamp

    step['timestamp'] = timestamp
    step['seq'] = int(msg.images[0].header.frame_id)

    # Image
    cv_img = bridge.compressed_imgmsg_to_cv2(msg.images[0], desired_encoding="passthrough")
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGRA2RGB)
    # step['img'] = cv_img
    step['action_taken'] = {'action': -1, 'probability': -1}

    # Results
    step['results'] = []

    for r in msg.results:
        step['results'].append({'action': r.class_label, 'probability': r.class_prob})
        if r.class_prob > step['action_taken']['probability']:
            step['action_taken'] = {'action': r.class_label, 'probability': r.class_prob}

    steps_data['steps'].append(step)


In [57]:
df = pd.json_normalize(steps_data['steps'])
steps = df['seq'].max() - df['seq'].min()
print("Loaded {} steps.".format(steps))
print("Duration {:.2f} seconds".format(df['timestamp'].max()))
print("Average FPS: {:.1f}".format(steps / df['timestamp'].max()))

Loaded 1046 steps.
Duration 34.85 seconds
Average FPS: 30.0


In [53]:
action_analysis = df[['timestamp','seq','action_taken.action','action_taken.probability']]
display(action_analysis)

,timestamp,seq,action_taken.action,action_taken.probability
0,0.000000,126314,1,0.725098
1,0.028482,126315,1,0.721680
2,0.061664,126316,1,0.714355
3,0.096733,126317,1,0.714844
4,0.129739,126318,1,0.720703
...,...,...,...,...
765,34.720185,127356,8,0.217163
766,34.752211,127357,8,0.222046
767,34.785455,127358,8,0.227539
768,34.821155,127359,8,0.224121
